In [1]:
!pip install utils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=327c76ed12649f0a77019fe995223b74d977847b0ccefdc380457af7973fc0f0
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [4]:
import requests
import regex as re
import wget
import os

In [5]:
import numpy as np
from pathlib import Path
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
import xarray as xr
import datetime
import pandas as pd

In [6]:

sd = datetime.datetime(2023,4,14).strftime("%Y%m%d")
ed = datetime.datetime(2023,5,13).strftime("%Y%m%d")

In [7]:
ls=pd.date_range(sd,ed)

In [8]:
dates=[dt.strftime("%Y%m%d") for dt in ls]

In [9]:
years={date[:4] for date in dates}
months={date[4:6] for date in dates}

In [10]:
txt=""
for year in years:
    for month in months:
        base='https://www.ngdc.noaa.gov/dscovr/data/'+year+'/'+month+'/'
        txt+=requests.get(base).text

In [11]:
urllist=[]

In [12]:
for date in dates:
    base='https://www.ngdc.noaa.gov/dscovr/data/'+date[:4]+'/'+date[4:6]+'/'
    x=re.search(str("oe_m1m_dscovr_s"+date+"000000_"), txt).start()
    url=base+txt[x: x+71]
    print(date,": ",url)
    urllist.append(url[46:])
    wget.download(url)

20230414 :  https://www.ngdc.noaa.gov/dscovr/data/2023/04/oe_m1m_dscovr_s20230414000000_e20230414235959_p20230415030607_pub.nc.gz
20230415 :  https://www.ngdc.noaa.gov/dscovr/data/2023/04/oe_m1m_dscovr_s20230415000000_e20230415235959_p20230416021755_pub.nc.gz
20230416 :  https://www.ngdc.noaa.gov/dscovr/data/2023/04/oe_m1m_dscovr_s20230416000000_e20230416235959_p20230417021758_pub.nc.gz
20230417 :  https://www.ngdc.noaa.gov/dscovr/data/2023/04/oe_m1m_dscovr_s20230417000000_e20230417235959_p20230418021823_pub.nc.gz
20230418 :  https://www.ngdc.noaa.gov/dscovr/data/2023/04/oe_m1m_dscovr_s20230418000000_e20230418235959_p20230419021721_pub.nc.gz
20230419 :  https://www.ngdc.noaa.gov/dscovr/data/2023/04/oe_m1m_dscovr_s20230419000000_e20230419235959_p20230420021759_pub.nc.gz
20230420 :  https://www.ngdc.noaa.gov/dscovr/data/2023/04/oe_m1m_dscovr_s20230420000000_e20230420235959_p20230422014049_pub.nc.gz
20230421 :  https://www.ngdc.noaa.gov/dscovr/data/2023/04/oe_m1m_dscovr_s20230421000000_e2

In [13]:
l=len(urllist)

In [14]:
magdata=[]

In [15]:
for i in range(0,l):
  ds=xr.open_dataset('/content/'+urllist[i])
  df=ds.to_dataframe()
  actualdf=df.dropna()
  df_len=len(actualdf)
  for j in range(0,df_len,180):
   bx_gse_hour=np.nanmean(np.array(actualdf['bx_gse'][j:j+180]))
   by_gse_hour=np.nanmean(np.array(actualdf['by_gse'][j:j+180]))
   bz_gse_hour=np.nanmean(np.array(actualdf['bz_gse'][j:j+180]))
   b_gse=((bx_gse_hour)**2+(by_gse_hour)**2+(bz_gse_hour)**2)**0.5
   if bx_gse_hour+by_gse_hour+bz_gse_hour<0:
    magdata.append(-b_gse)
   else:
    magdata.append(b_gse)

In [16]:
magdata_numpy=np.array(magdata)

In [17]:
df=open('/content/drive/MyDrive/kp_index.txt', "r")
lines = df.readlines()
forecastdata=[]
for index, line in enumerate(lines):
      lines[index] = line.strip().split()
      l=len(lines[index])
      for i in range(l):
        forecastdata.append(float(lines[index][i]))

In [18]:
fnumpy=np.array(forecastdata)

In [19]:
import joblib

In [20]:
import warnings
warnings.filterwarnings('ignore')

In [21]:
from sklearn.linear_model import LinearRegression

In [22]:
regr=LinearRegression().fit(magdata_numpy.reshape(-1,1),fnumpy.reshape(-1,1))

In [23]:
regr.coef_

array([[-0.06089302]])

In [24]:
regr.intercept_

array([2.42220145])

In [29]:
lm=len(magdata)
acc=0
for i in range(lm):
  predval=regr.predict(np.array([magdata[i]]).reshape(-1,1))
  if predval-fnumpy[i]<1.2:
    acc+=1
print("Accuracy:"+str((acc/lm)*100))

Accuracy:86.25


In [ ]:
joblib.dump(regr,"/content/drive/MyDrive/regr.pkl")